#1 Import necessary libraries

In [1]:
import sqlite3
import pandas as pd

#2 Create a connection

In [2]:
con = sqlite3.connect("cademycode.db")

In [ ]:
data_base = pd.read_sql_query("SELECT * FROM sqlite_master", con)
data_base.head()

,type,name,tbl_name,rootpage,sql
0,table,cademycode_students,cademycode_students,2,CREATE TABLE cademycode_students (\n\tuuid INT...
1,table,cademycode_courses,cademycode_courses,5,CREATE TABLE cademycode_courses (\n\tcareer_pa...
2,table,cademycode_student_jobs,cademycode_student_jobs,6,CREATE TABLE cademycode_student_jobs (\n\tjob_...


In [8]:
students = pd.read_sql_query("SELECT * FROM cademycode_students", con)
students.head(10)

,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
0,1,Annabelle Avery,1943-07-03,F,"{""mailing_address"": ""303 N Timber Key, Irondal...",7.0,6.0,1.0,4.99
1,2,Micah Rubio,1991-02-07,M,"{""mailing_address"": ""767 Crescent Fair, Shoals...",7.0,5.0,8.0,4.4
2,3,Hosea Dale,1989-12-07,M,"{""mailing_address"": ""P.O. Box 41269, St. Bonav...",7.0,8.0,8.0,6.74
3,4,Mariann Kirk,1988-07-31,F,"{""mailing_address"": ""517 SE Wintergreen Isle, ...",6.0,7.0,9.0,12.31
4,5,Lucio Alexander,1963-08-31,M,"{""mailing_address"": ""18 Cinder Cliff, Doyles b...",7.0,14.0,3.0,5.64
5,6,Shavonda Mcmahon,1989-10-15,F,"{""mailing_address"": ""P.O. Box 81591, Tarpon Sp...",6.0,10.0,3.0,10.12
6,7,Terrell Bleijenberg,1959-05-05,M,"{""mailing_address"": ""P.O. Box 53471, Oskaloosa...",2.0,9.0,8.0,24.17
7,8,Stanford Allan,1997-11-22,M,"{""mailing_address"": ""255 Spring Avenue, Point ...",3.0,3.0,1.0,19.54
8,9,Tricia Delacruz,1961-10-20,F,"{""mailing_address"": ""997 Dewy Apple, Lake Lind...",1.0,6.0,9.0,1.75
9,10,Regenia van der Helm,1999-02-23,N,"{""mailing_address"": ""220 Middle Ridge, Falcon ...",5.0,7.0,6.0,13.55


In [10]:
students.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uuid                    5000 non-null   int64 
 1   name                    5000 non-null   object
 2   dob                     5000 non-null   object
 3   sex                     5000 non-null   object
 4   contact_info            5000 non-null   object
 5   job_id                  4995 non-null   object
 6   num_course_taken        4749 non-null   object
 7   current_career_path_id  4529 non-null   object
 8   time_spent_hrs          4529 non-null   object
dtypes: int64(1), object(8)
memory usage: 351.7+ KB


In [16]:
students.dtypes

uuid                       int64
name                      object
dob                       object
sex                       object
contact_info              object
job_id                    object
num_course_taken          object
current_career_path_id    object
time_spent_hrs            object
dtype: object

In [15]:
students.isna().sum()

uuid                        0
name                        0
dob                         0
sex                         0
contact_info                0
job_id                      5
num_course_taken          251
current_career_path_id    471
time_spent_hrs            471
dtype: int64

In [20]:
students.describe()

,uuid
count,5000.000000
mean,2500.500000
std,1443.520003
min,1.000000
25%,1250.750000
50%,2500.500000
75%,3750.250000
max,5000.000000


In [ ]:
students.value_counts(dropna=True)

uuid                        0
name                        0
dob                         0
sex                         0
contact_info                0
job_id                      5
num_course_taken          251
current_career_path_id    471
time_spent_hrs            471
dtype: int64

In [15]:
courses = pd.read_sql_query("SELECT * FROM cademycode_courses", con)
courses.head()

,career_path_id,career_path_name,hours_to_complete
0,1,data scientist,20
1,2,data engineer,20
2,3,data analyst,12
3,4,software engineering,25
4,5,backend engineer,18


In [22]:
courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   career_path_id     10 non-null     int64 
 1   career_path_name   10 non-null     object
 2   hours_to_complete  10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 372.0+ bytes


In [23]:
courses.describe()

,career_path_id,hours_to_complete
count,10.00000,10.000000
mean,5.50000,21.900000
std,3.02765,6.707376
min,1.00000,12.000000
25%,3.25000,18.500000
50%,5.50000,20.000000
75%,7.75000,26.500000
max,10.00000,35.000000


In [24]:
courses.value_counts()

career_path_id  career_path_name           hours_to_complete
1               data scientist             20                   1
2               data engineer              20                   1
3               data analyst               12                   1
4               software engineering       25                   1
5               backend engineer           18                   1
6               frontend engineer          20                   1
7               iOS developer              27                   1
8               android developer          27                   1
9               machine learning engineer  35                   1
10              ux/ui designer             15                   1
Name: count, dtype: int64

In [16]:
student_jobs = pd.read_sql_query("SELECT * FROM cademycode_student_jobs", con)
student_jobs.head()

,job_id,job_category,avg_salary
0,1,analytics,86000
1,2,engineer,101000
2,3,software developer,110000
3,4,creative,66000
4,5,financial services,135000


In [25]:
student_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   job_id        13 non-null     int64 
 1   job_category  13 non-null     object
 2   avg_salary    13 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 444.0+ bytes


In [26]:
student_jobs.describe()

,job_id,avg_salary
count,13.000000,13.000000
mean,4.384615,89230.769231
std,2.662657,34727.879881
min,0.000000,10000.000000
25%,3.000000,66000.000000
50%,4.000000,86000.000000
75%,6.000000,110000.000000
max,9.000000,135000.000000


In [27]:
student_jobs.value_counts()

job_id  job_category        avg_salary
3       software developer  110000        2
4       creative            66000         2
5       financial services  135000        2
0       other               80000         1
1       analytics           86000         1
2       engineer            101000        1
6       education           61000         1
7       HR                  80000         1
8       student             10000         1
9       healthcare          120000        1
Name: count, dtype: int64